In [11]:
from recons import antenna_set, coincidence_set, PWF_loss
import numpy as np
import matplotlib.pyplot as plt
from emcee import EnsembleSampler


In [12]:
def logprob(angles, coords, times):
    return -0.5*PWF_loss(angles, coords, times)

In [18]:
def MCMC_minimizer(nwalkers, ndim, logprob, args):
    sampler = EnsembleSampler(nwalkers, ndim, logprob, args=args)
    print('sampler', sampler)
    #generates 10 values of theta in the range [90°, 180°]
    #and 10 values of phi in the range [180°, 360°]
    #(GRAND conventions)
    thetas = np.random.rand(10)*np.pi/2 + np.pi/2
    phis = np.random.rand(10)*2.*np.pi
    #concatenates theta et phi
    p0 = np.vstack((thetas, phis)).T
    state  = sampler.run_mcmc(p0, 100)
    #print('state',state)
    sampler.reset()
    sampler.run_mcmc(state, 10000)
    samples = sampler.get_chain(flat=True, thin=25)
    logprobs = sampler.get_log_prob(flat=True, thin=25)
    mask= (samples[:,0]>np.pi/2)*(samples[:,0]<np.pi)*(samples[:,1]>0)*(samples[:,1]<2*np.pi)
    med = np.median(samples[mask,:], axis=0)
    return med
an = antenna_set('/Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/coord_antennas.txt')
co = coincidence_set('/Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/Rec_coinctable.txt', an)
current_recons = 0
args=(co.antenna_coords_array[current_recons,:],co.peak_time_array[current_recons,:])
print(MCMC_minimizer(10,2, logprob, args))

 Reading antenna positions from file /Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/coord_antennas.txt
 Reading coincidence(s): index, peak time, peak amplitude from file /Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/Rec_coinctable.txt
sampler <emcee.ensemble.EnsembleSampler object at 0x16d33b0d0>
[2.47291534 3.14064961]
